<a href="https://colab.research.google.com/github/duotien/mono_pitch_tracker/blob/main/notebooks/mono_pitch_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [3]:
%%capture
%git clone --recursive https://github.com/duotien/mono_pitch_tracker.git
%apt -qq install -y sox
%pip install pyyaml==5.4.1 sox librosa==0.9.2 torchinfo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(f'Get file from google drive...')
import shutil
file_to_copy_path = '/content/drive/MyDrive/DataBase/MedleyDB/V1_MIX_AUDIO_ONLY.zip'
file_to_paste_path = '/content/V1_MIX_AUDIO_ONLY.zip'

shutil.copy2(file_to_copy_path, file_to_paste_path)

In [ ]:
%%capture
!unzip -o V1_MIX_AUDIO_ONLY.zip -d /content/mono_pitch_tracker/medleydb/medleydb/data/Audio/

# Import

In [ ]:
import sys
import os

NOTEBOOK_DIR = os.getcwd()
MONO_PITCH_TRACKER_DIR = os.path.join(os.getcwd(), 'mono_pitch_tracker/')
MEDLEYDB_DIR = os.path.join(MONO_PITCH_TRACKER_DIR, 'medleydb/')

sys.path.append(MONO_PITCH_TRACKER_DIR)
sys.path.append(MEDLEYDB_DIR)

In [ ]:
# change working directory to `mono_pitch_tracker/``
%cd $MONO_PITCH_TRACKER_DIR

In [ ]:
import numpy as np
import pandas as pd
from torchinfo import summary
import yaml

import medleydb
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils.files import get_file_name, list_file_paths_in_dir, list_folder_paths_in_dir

# from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.medleydb_melody import gen_label
from pitch_tracker.utils import dataset

/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


# Prepare Dataset

In [ ]:
# Prepare datset
!python scripts/prepare_dataset.py

# Modeling & training

In [ ]:
# create config file
p = {
    # dataset
    'batch_size': 16,
    # fit
    'n_epochs': 100,
    'learning_rate': 1e-3,
    # early stopping
    'es_patience': 10,
    'es_verbose': True,
    'es_dir_path': './checkpoints',
    # lr scheduler
    'ls_patience': 6,
    'ls_factor': 0.2,
    # misc
    'device': 'DEVICE',
}

with open('./scripts/config/model_config.yml', 'w') as f:
    yaml.dump(p,f,sort_keys=False)

In [ ]:
# Train model
! python scripts/model.py

In [ ]:
model = torch.load('/content/drive/MyDrive/School work/KLTN/2022-2023/models/mpt_v01_20231028-2010.pth')

In [ ]:
import torch

dummy_in = torch.randn(8, 1, 1050, 88)
in_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
out_names = [ "output1" ]

torch.onnx.export(model, dummy_in, f="model.onnx", input_names=in_names, output_names=out_names, opset_version=7, verbose=True)